In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

In [ ]:
!unzip fracture_datset.zip


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\Users\Riddhi\Documents\DIP - datasets\Wrist_Dataset\archive(6)'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [3]:
train_path = r'C:\Users\Riddhi\Documents\DIP - datasets\Wrist_Dataset\archive (6)\train'
test_path = r'C:\Users\Riddhi\Documents\DIP - datasets\Wrist_Dataset\archive (6)\val'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [6]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 7, 7, 1536)       10783535  
                                                                 
 gaussian_noise (GaussianNoi  (None, 7, 7, 1536)       0         
 se)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                        

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall','AUC'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lrp = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)
filepath = 'best_model.h5'

# Modify the monitor value of ModelCheckpoint callback
checkpoint_monitor = 'val_loss'  # Use a different metric that doesn't involve tensors

checkpoint = ModelCheckpoint(filepath, monitor=checkpoint_monitor, verbose=1, save_best_only=True,mode='min',save_weights_only=True )
call = [checkpoint, lrp]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=50,
    callbacks=call
)


Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 1.0348 - accuracy: 0.5900 - precision: 0.6162 - recall: 0.5810 - auc: 0.6445
Epoch 1: val_loss improved from inf to 0.70624, saving model to best_model.h5
50/50 [==============================] - 129s 2s/step - loss: 1.0348 - accuracy: 0.5900 - precision: 0.6162 - recall: 0.5810 - auc: 0.6445 - val_loss: 0.7062 - val_accuracy: 0.7283 - val_precision: 0.5970 - val_recall: 0.9875 - val_auc: 0.8055 - lr: 0.0010
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.7738 - accuracy: 0.6800 - precision: 0.7009 - recall: 0.7009 - auc: 0.7368
Epoch 2: val_loss did not improve from 0.70624
50/50 [==============================] - 106s 2s/step - loss: 0.7738 - accuracy: 0.6800 - precision: 0.7009 - recall: 0.7009 - auc: 0.7368 - val_loss: 0.8104 - val_accuracy: 0.5900 - val_precision: 0.4916 - val_recall: 0.7333 - val_auc: 0.6443 - lr: 0.0010
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.7

In [ ]:
model.evaluate(train_generator)

2216/2216 [==============================] - 1173s 529ms/step - loss: 0.3673 - accuracy: 0.8353 - precision: 0.8109 - recall: 0.8697 - auc: 0.9185


[0.36730077862739563,
 0.8352702260017395,
 0.8108912706375122,
 0.869723916053772,
 0.9184583425521851]

In [ ]:
model.evaluate(validation_generator)

150/150 [==============================] - 79s 526ms/step - loss: 0.6895 - accuracy: 0.6883 - precision: 0.6047 - recall: 0.6375 - auc: 0.7655


[0.6895389556884766,
 0.6883333325386047,
 0.6047430634498596,
 0.637499988079071,
 0.765532374382019]

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
img = load_img('C:\Users\Riddhi\Documents\DIP - datasets\Wrist_Dataset\archive (6)\val\not fractured\1-rotated3.jpg',target_size=(224,224))
imag = img_to_array(img)
imaga = np.expand_dims(imag,axis=0)
ypred = model.predict(imaga)
print(ypred)
a=ypred[0]
if a<0.5:
      op="Fracture"
else:
      op="Normal"
plt.imshow(img)
print("THE UPLOADED X-RAY IMAGE IS: "+str(op))

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
#from pytesseract import pytesseract
import cv2
from matplotlib import pyplot as plt

def browse_image():
    file_path = filedialog.askopenfilename(filetypes=(("Image files", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")))
    if file_path:
        image = Image.open(file_path)
        image.thumbnail((400, 400))
        photo = ImageTk.PhotoImage(image)
        image_label.configure(image=photo)
        image_label.image = photo

def classify_image():
    a=ypred[0]
    if a<0.5:
        result_label.configure(text="Fracture Detected")
    else:
        result_label.configure(text="No Fracture Detected")
            
window = tk.Tk()
window.title("Fracture Detection")
window.geometry("600x400")
left_frame = tk.Frame(window)
left_frame.pack(side=tk.LEFT, padx=20, pady=20)
load_button = tk.Button(left_frame, text="Load Image", command=browse_image)
load_button.pack(pady=10)
classify_button = tk.Button(left_frame, text="Classify Image", command=classify_image)
classify_button.pack(pady=10)


right_frame = tk.Frame(window)
right_frame.pack(side=tk.RIGHT, padx=20, pady=20)


image_label = tk.Label(right_frame)
image_label.pack()


result_label = tk.Label(window, text="")
result_label.pack(pady=10)
window.mainloop()


In [5]:
from tkinter import filedialog
from PIL import Image, ImageTk
import tkinter as tk
import numpy as np
import cv2

def browse_image():
    file_path = filedialog.askopenfilename(filetypes=(("Image files", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")))
    if file_path:
        image = Image.open(file_path)
        image.thumbnail((400, 400))
        photo = ImageTk.PhotoImage(image)
        image_label.configure(image=photo)
        image_label.image = photo

def preprocess_image():
    image = image_label.image
    image_array = np.array(image)
    processed_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    processed_image = cv2.GaussianBlur(processed_image, (5, 5), 0)
    
    processed_image_label.configure(image=ImageTk.PhotoImage(Image.fromarray(processed_image)))
    processed_image_label.image = processed_image_label.image
    
def classify_image():
    result_label.configure(text="Classifying Image...")

window = tk.Tk()
window.title("Fracture Detection")
window.geometry("800x400")

left_frame = tk.Frame(window)
left_frame.pack(side=tk.LEFT, padx=20, pady=20)

load_button = tk.Button(left_frame, text="Load Image", command=browse_image)
load_button.pack(pady=10)

preprocess_button = tk.Button(left_frame, text="Preprocess Image", command=preprocess_image)
preprocess_button.pack(pady=10)

classify_button = tk.Button(left_frame, text="Classify Image", command=classify_image)
classify_button.pack(pady=10)

right_frame = tk.Frame(window)
right_frame.pack(side=tk.RIGHT, padx=20, pady=20)

image_label = tk.Label(right_frame)
image_label.pack(side=tk.LEFT)

processed_image_label = tk.Label(right_frame)
processed_image_label.pack(side=tk.LEFT)

result_label = tk.Label(window, text="")
result_label.pack(pady=10)

window.mainloop()
